#All

#####Import library

In [ ]:
import numpy as np
import pandas as pd
import os

from IPython.display import Image
from PIL import Image
from scipy.signal import correlate2d
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#####Read file

In [ ]:
df = pd.read_csv(r"30.csv")
print(df)

#####find uniqe sub_id

In [ ]:
# Extract the unique IDs from the 'sub_id' column
unique_ids = df['sub_id'].unique()

# Print the unique IDs
for unique_id in unique_ids:
    print(unique_id)

####how many uniqe id buy package and save data

#####count every user has how many pkg purchase

In [ ]:
# Count the occurrences of each unique ID
id_counts = df['sub_id'].value_counts()

# Print the ID counts
for unique_id, count in id_counts.items():
    print(f"Unique ID: {unique_id}, Count: {count}")

##### plot data

In [ ]:
# Plotting the histogram
plt.figure(figsize=(10, 6))
plt.hist(id_counts, bins=100, alpha=0.75, color='blue')

# Customize the plot
plt.title('Count of Unique IDs')
plt.xlabel('Count')
plt.ylabel('Frequency')

# Display the plot
plt.show()

##### create csv file for every unique id

In [ ]:
import shutil

folder_path = r'D:\Daneshgah\Az Shabake\unique_id_behavior'

# Use the shutil.rmtree() function to remove the entire folder and its contents
shutil.rmtree(folder_path)

#first delete
# !rm -r 'unique_id_behavior'
#then create folder
# Create the folder if it doesn't exist
folder_name = 'unique_id_behavior'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Create a dictionary to store lists for each unique ID
id_lists = {unique_id: [] for unique_id in id_counts.index}

# Iterate over each row and append to the corresponding list
for index, row in df.iterrows():
    sub_id = row['sub_id']
    id_lists[sub_id].append(row.tolist())

minimum_data_needed =15
# Save each list as a separate CSV file in the folder
for unique_id, rows_list in id_lists.items():
    if len(rows_list) >= minimum_data_needed:
        # Create a DataFrame from the list of rows
        sub_df = pd.DataFrame(rows_list, columns=df.columns)

        # Sort the DataFrame based on the "actvn_dt" column
        sub_df = sub_df.sort_values(by='actvn_dt')

        # Add a new column "times" with incrementing values starting from 1
        sub_df['times'] = range(1, len(sub_df) + 1)

        # Save the DataFrame as a CSV file in the folder
        file_path = os.path.join(folder_name, f'{unique_id}_30.csv')
        sub_df.to_csv(file_path, index=False)
        print(f'Saved rows for Unique ID {unique_id} in {file_path}')
    else:
        print(f'Skipped saving rows for Unique ID {unique_id} (less than {minimum_data_needed} rows)')

##### make it zip file

In [ ]:
import zipfile
# Create a zip file of the folder
zip_file_name = 'unique_id_behavior.zip'
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(folder_name):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.basename(file_path))

print(f'Successfully created {zip_file_name}.')

#### train and predict last bought

#####predict next pkg linear reg

In [ ]:
from word2number import w2n
import inflect

def words_to_number(words):
    try:
        number = w2n.word_to_num(words)
        return number
    except ValueError as e:
        print(f"Error: {e}")
        return None

def number_to_words(number):
    p = inflect.engine()
    return p.number_to_words(number)

In [ ]:
# Function to predict the next package using a decision tree classifier
def predict_next_pkg_log_reg(X_train, y_train, next_time, last_pkg_bought):
    y_train_word = []
    for i in range(len(y_train)):
       y_train_word.append(number_to_words(y_train[i]))

    # Feature scaling using StandardScaler
    sc_X = StandardScaler()
    X_train_scaled = sc_X.fit_transform(X_train)

    # # Encode the labels with OneHotEncoder
    # encoder = OneHotEncoder(sparse=False)
    # encoded_features = encoder.fit_transform(y_train.reshape(-1, 1))

    # Fit the logistic reg classifier
    lr_clf = LogisticRegression()
    lr_clf.fit(X_train_scaled, y_train_word)

    # Perform scaling on the next time data
    next_time_scaled = sc_X.transform(np.array([next_time]))


    # Predict the features
    predicted_features = lr_clf.predict(next_time_scaled)
    predicted_pkg =words_to_number(str(predicted_features[0]))

    print(predicted_features)
    # Inverse transform the one-hot encoded matrix to get the predicted label
    if np.all(predicted_features == 0):
        # Handle the case where all zeros are predicted
        decoded_labels = "No label predicted"

    return predicted_pkg


#####Read files

In [ ]:
csv_files = [file for file in os.listdir(folder_name) if file.endswith('.csv')]
real_data = []
predict_next_pkg_data = []
counter_noise = 0
# Read each CSV file one by one
columns_x = [2,3,4,5,6,7,10,17,25, 21,22,23,24]
# 18,25
for file in csv_files:
    # Construct the file path
    file_path = os.path.join(folder_name, file)

    # Read the CSV file
    df = pd.read_csv(file_path)

    # Get information
    X_train = df.iloc[:, columns_x].values
    y_train = df["voice_pkg_code"].values.reshape(-1, 1)
    df[['prod_stat_eng', 'gender', 'Pk_ATL/BTL', 'device_type', 'Pk_regular/Event']] = df[['prod_stat_eng', 'gender', 'Pk_ATL/BTL', 'device_type', 'Pk_regular/Event']].apply(LabelEncoder().fit_transform)
    X_train = X_train[:-1]
    y_train = np.delete(y_train, -1)
    sub_id = df["sub_id"].iloc[-1]
    next_time = df.iloc[-1, columns_x].values
    last_pkg_bought = df["voice_pkg_code"].iloc[-1]
    
    try:
    # Predict
        decoded_labels = predict_next_pkg_log_reg(X_train, y_train, next_time,last_pkg_bought)
        print(decoded_labels)
        predict_next_pkg_data.append(int(decoded_labels))
        real_data.append(df["voice_pkg_code"].iloc[-1])

    except:
        counter_noise = counter_noise + 1


In [ ]:
print("accuracy_score = ",accuracy_score(real_data,predict_next_pkg_data))
print(confusion_matrix(real_data, predict_next_pkg_data))

In [ ]:
list_features =[]
accuracy_list = []

n_features = X_train.shape[1]
# print(n_features)
selected_features = np.ones(n_features, dtype=bool)
# print(selected_features)
csv_files = [file for file in os.listdir(folder_name) if file.endswith('.csv')]
initial_accuracy = 0.2
for i in range(n_features - 1, 0, -1):
    # Create a copy of selected_features, so we don't modify the original array
    mask = np.copy(selected_features)
    mask[i] = False
    real_data = []
    predict_next_pkg_data = []
    counter_noise = 0
    # Read each CSV file one by one
    columns_x = [2,3,4,5,6,7,10,17,25, 21,22,23,24]
    for file in csv_files:
        # Construct the file path
        file_path = os.path.join(folder_name, file)
        # Read the CSV file
        df = pd.read_csv(file_path)

        # Get information
        X_train = df.iloc[:, columns_x].values
        next_time = df.iloc[-1, columns_x].values

        # Select the subset of features based on the mask
        X_train_subset = X_train[:, mask]
        next_time_subset = next_time[mask]

        y_train = df["voice_pkg_code"].values.reshape(-1, 1)
        X_train = X_train[:-1]
        y_train = np.delete(y_train, -1)
        sub_id = df["sub_id"].iloc[-1]
        last_pkg_bought = df["voice_pkg_code"].iloc[-1]
        # print(X_train)
        try:
        # Predict
            decoded_labels = predict_next_pkg_log_reg(X_train_subset, y_train, next_time_subset,last_pkg_bought)
            print(decoded_labels)
            predict_next_pkg_data.append(int(decoded_labels))
            real_data.append(df["voice_pkg_code"].iloc[-1])

        except:
            counter_noise = counter_noise + 1
    
    # Evaluate the performance of the model on the test set
    accuracy_subset = accuracy_score(real_data, predict_next_pkg_data)
    list_features.append(np.where(mask)[0])
    accuracy_list.append(accuracy_subset)
    # If the accuracy improves, update the selected features
    if accuracy_subset >= initial_accuracy:
        selected_features = mask
        initial_accuracy = accuracy_subset

# Final selected features
selected_features_indices = np.where(selected_features)[0]
print("Final selected features indices:", selected_features_indices)

In [ ]:
print(list_features)
print(accuracy_list)

In [ ]:
# Plot the histograms
plt.figure(figsize=(10, 5))

plt.hist(predict_next_pkg_data, bins=160, color='blue', alpha=0.5, label='predict')
plt.hist(real_data, bins=160, color='red', alpha=0.5, label='last bought')

plt.xlabel('voice_pkg_code')
plt.ylabel('Frequency')
plt.title('predict with decision tree and ')
plt.legend()

plt.tight_layout()
plt.show()

